
# Semestral Work Assignment (4IZ172) – Clustering


# Imports

## Libraries

In [65]:
# pandas
import pandas as pd

# chardet - used for the detection of the encoding of the file
import chardet

# sklearn
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# string
import string

# re for regular expressions
import re

# nltk for
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# inflect 
import inflect

# visualization
import matplotlib.pyplot as plt
import seaborn as sns



## Dataset
For this coursework i chose the FR aerospace dataset.

### Checking the encoding and loading

In [66]:
path="data/FR-aerospace-translate.csv"
with open(path, 'rb') as f:
    rawdata = f.read()
result = chardet.detect(rawdata)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

# We have the encoding, now we can read the CSV file
df_aero = pd.read_csv("data/FR-aerospace-translate.csv",encoding=encoding)
df_aero = df_aero.dropna()
df_aero

Detected encoding: utf-8


,Name,URL,FR,EN
0,ACPI SARL,https://acpi.tech,"Société d’ingénierie fondée en 1994, nous somm...","Established in 1994, we specialize in the foll..."
1,ACRI-ST,https://www.acri-st.fr/fr/,Le siege d’ACRI-ST (créée en 1989) est situé a...,ACRI-ST's head office (created in 1989) is loc...
2,AD INDUSTRIES HYDRAULICS,https://adgroupe.com/,Filiale du Groupe AD Industrie (CA 190 M - 1 5...,Subsidiary of the AD Industry Group (CA 190 M ...
3,AD-WAIBE,https://ad-waibe.fr,Savoir-faire : - Expertise Design Thinking et ...,Know how : - Design Thinking and innovation ex...
4,ADAGOS,https://adagos.com,ADAGOS a développé une nouvelle approche neuro...,To address the challenges in the implementatio...
...,...,...,...,...
600,AES,https://ae-services.fr,Spécialisée dans l’ingénierie d’équipements d’...,"AES, specialized in the engineering of automat..."
601,AGUILA TECHNOLOGIES,https://aguila.fr/,AGUILA est un acteur clé des technologies comm...,AGUILA Technologies is an independent high-tec...
602,AGENIUM,https://agenium.group/fr,AGENIUM accompagne les grands comptes de la Dé...,AGENIUM a key partner for major accounts of De...
603,AEVA,https://aece-group.com/en/entity/aeva/,"AEVA, groupe AECE, compte 70 collaborateurs su...","AEVA, AECE Group, has 70 employees at Fléac cl..."



# Preprocessing
The section should contain:

Description of preprocessing steps involved
Report on the preprocessing results


In [67]:
documents = df_aero["EN"].tolist()
names = df_aero["Name"].tolist()
vectorizer = TfidfVectorizer(stop_words='english')
documents


['Established in 1994, we specialize in the following fields: - Industrial Computing: Computing, Databases, MES and Supervisor, Traceability, Cloud (AZURE), IOT LPWAN (Lorawan) - Test Bench: Aeronautics, Electrical Equipment - Traceability - Process Automatism: Production by "batch", fuzzy logic control - Industrial Organisation Audit: Flux, Obsolescence Computing and Automation Our Mechanical Activity can also deploy robotics solutions, conventional or collaborative. Under certification EN9100 and ISO 9001 Our partnerships: - MICROSOFT PARTNER NETWORK , AZURE - Customer System SIEMENS - AVEVA systems Integrator : WONDERWARE - Specialist History SCHNEIDER - Specialist History CITECT SCADA',
 "ACRI-ST's head office (created in 1989) is located in Sophie Antipolis. In France, ACRI-ST also has 3 establishments located in Paris, Toulouse and Grasse (the Center d'Etudes et de Recherche Grasse ACRI-ST, CERGA). Its team of engineers and researchers is specialized in space remote sensing (Eart

In [ ]:
dtm = vectorizer.fit_transform(documents)
similarity_matrix = cosine_similarity(dtm, dtm)

print(similarity_matrix.shape)  

(556, 556)



# Clustering
This section should contain:

List of clustering methods considered
Estimation of the number of clusters (methods, results, discussion)
Comparison of (the results of) clustering methods on the selected dataset



# Cluster Descriptions
The clusters (at least important part of them) should be described.

This section should contain:

Description of the selected approach
Examples of the descriptions obtained



# Visualization
This section should contain:

At least one visualization related to clustering



# Analytical Report
The entire work should be ended by the final analytical report of findings on the selected dataset. It should point out your main results, their justifications, hypotheses, conclusions, outliers, ...